In [56]:
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

In [57]:
paths1 = glob("/Users/phamhoang1408/Desktop/self-instruct/rewrite_query/query_question/*")
data = []
for path in paths1:
    with open(path, "r") as f:
        data.extend(json.load(f))
paths2 = glob("/Users/phamhoang1408/Desktop/self-instruct/rewrite_query/query_question2/*")
for path in paths2:
    temp_paths = glob(path + "/*.json")
    for temp_path in temp_paths:
        with open(temp_path, "r") as f:
            data.extend(json.load(f))

ds = Dataset.from_list(data)
ds.shuffle()[0]

{'history': [{'content': 'Có các sự kiện văn hóa hoặc lễ hội nào diễn ra ở Pará trong suốt năm không?',
   'role': 'user'},
  {'content': 'Có, có một số sự kiện văn hóa và lễ hội diễn ra ở Pará trong suốt năm. Dưới đây là một số sự kiện đáng chú ý:\n\n1. Festival de Ópera do Theatro da Paz - Đây là một lễ hội opera nổi tiếng diễn ra ở Belem vào tháng Ba hàng năm.\n\n2. Círio de Nazaré - Đây là một trong những lễ hội tôn giáo lớn nhất ở Brazil và diễn ra ở Belem vào tháng Mười.\n\n3. Festa do Çairé - Lễ hội này diễn ra ở Santarém và là một dịp để kỷ niệm văn hóa bản địa với những điệu nhảy, âm nhạc và đồ ăn.\n\n4. Festival de Carimbo - Đây là một lễ hội hàng năm ở Belem, nơi các nghệ sĩ địa phương biểu diễn âm nhạc carimbo, một thể loại âm nhạc truyền thống của Pará.\n\n5. Arraial de Belo - Đây là một lễ hội truyền thống ở Belo, nơi mọi người nhảy múa và hát theo các bài hát của âm nhạc dân gian Brazil.\n\n6. Festival de Parintins - Đây là một lễ hội Amazon diễn ra vào tháng Sáu ở thành

In [58]:
ds.shuffle()[0]

{'history': [{'content': 'Những yếu tố nào đã góp phần vào sự phát triển của Phục hưng ở châu Âu và nó đã ảnh hưởng như thế nào đến nghệ thuật, văn học và khoa học?',
   'role': 'user'},
  {'content': 'Phục hưng là một giai đoạn tái sinh văn hóa đã xảy ra ở châu Âu trong thế kỷ 14 đến 17 sau Công nguyên. Nó được đặc trưng bởi sự chuyển đổi từ tư tưởng trung cổ sang hiện đại và được đánh dấu bởi sự quan tâm lại đến văn hóa cổ điển, giáo dục và nghệ thuật. Những yếu tố sau đã góp phần vào sự phát triển của Phục hưng ở châu Âu:\n\n1. Nhân văn học: Phong trào Nhân văn học đã truyền cảm hứng cho sự phát triển của Phục hưng. Triết lý này khuyến khích việc nghiên cứu văn học cổ điển, giáo dục và nghệ thuật như một cách để hiểu về bản chất con người và thế giới.\n\n2. Thương mại và buôn bán: Sự gia tăng thương mại và buôn bán trong thời kỳ Phục hưng đã tạo cơ hội cho việc trao đổi ý tưởng, kiến thức và vật liệu văn hóa từ các vùng khác nhau.\n\n3. Bảo trợ: Các bậc bảo trợ giàu có, như gia đình

In [59]:
TEMPLATE = """Dưới đây là lịch sử trò chuyện cho đến nay, và một câu hỏi mới được đặt ra bởi người dùng. Dựa vào lịch sử trò chuyện và câu hỏi mới, hãy tạo ra một câu hỏi hoàn chỉnh chứa thông tin ngữ cảnh của văn bản để tôi có thể trả lời mà không cần xem lại lịch sử trò chuyện.
Lịch sử trò chuyện:
{history_text}Câu hỏi mới từ #User: {new_question}"""
def mapper(x):
    history = x['history'][:-2]
    history_text = ""
    if len(history) == 0:
        history_text = "Không có lịch sử\n"
    for h in history:
        if h['role'] == 'user':
            history_text += "#USER " + h['content'] + "\n"
        elif h['role'] == 'assistant':
            history_text += "#BOT " + h['content'] + "\n"
    new_question = x['history'][-2]['content']
    prompt = TEMPLATE.format(history_text=history_text, new_question=new_question)
    try:
        return {
            'messages': [
                {'role': 'user', 'content': prompt},
                {'role': 'assistant', 'content': eval(x['generated_question'])['question']}
            ]
        }
    except:
        return {
            'messages':[]
        }
ds = ds.map(mapper,remove_columns=['generated_question','history'])
ds = ds.filter(lambda x: len(x['messages']) > 0)

Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [65]:
temp = ds.shuffle()[0]
print(temp['messages'][0]['content'])
print(temp['messages'][1]['content'])

Dưới đây là lịch sử trò chuyện cho đến nay, và một câu hỏi mới được đặt ra bởi người dùng. Dựa vào lịch sử trò chuyện và câu hỏi mới, hãy tạo ra một câu hỏi hoàn chỉnh chứa thông tin ngữ cảnh của văn bản để tôi có thể trả lời mà không cần xem lại lịch sử trò chuyện.
Lịch sử trò chuyện:
#USER Điều này thật tuyệt vời, tôi đã quên mất. Trước đây có những cuốn sách nấu ăn chỉ dành riêng cho việc này, chúng cung cấp cho bạn thời gian và quãng đường để nấu ăn.
Nhưng điều duy nhất tôi muốn bổ sung là hãy chắc chắn rằng bạn không có bất kỳ rò rỉ dầu, nhiên liệu hoặc khí thải nào, nếu không bạn sẽ không thích mùi vị của bữa trưa của mình. Tin tôi đi, tôi là một thợ máy và đã ăn quá nhiều bánh sandwich trong sự vội vã với đôi tay dầu mỡ.
Bạn có một ý kiến đúng. Vì lý do đó, có thể tốt hơn là bỏ qua bình luận của tôi về việc để thức ăn trên động cơ trong một thời gian sau khi tắt máy, vì thức ăn sẽ bắt đầu hút không khí trở lại khi nó nguội dần, dù chậm. Bạn có thể đưa ra một gợi ý khác để để thứ

In [64]:
ds.to_json('./final_dataset.jsonl', force_ascii=False)

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

3784534